<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
##### Your Code Here #####
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy
# fix random seed for reproducibility
numpy.random.seed(45)
pd.set_option('display.max_columns', 500)

c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,

# Loading and Preprocessing Data

In [2]:
url = "WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
samples = df.shape[0]

oe = OrdinalEncoder()
df_transformed = oe.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df_transformed[:, :-1], df_transformed[:, -1], 
                                                    test_size=0.2, random_state=45)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5634, 20) (1409, 20) (5634,) (1409,)


# Baseline Model

In [4]:
inputs = X_train.shape[1]

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Fit Model
model.fit(X_train, y_train, 
          validation_split=0.33, 
          epochs=10,
          batch_size=64
        )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3774 samples, validate on 1860 samples
Epoch 1/10
3774/3774 [==============================] - 1s 204us/sample - loss: 30.4078 - acc: 0.6198 - val_loss: 4.0367 - val_acc: 0.7059
Epoch 2/10
3774/3774 [==============================] - 0s 87us/sample - loss: 3.6568 - acc: 0.6330 - val_loss: 2.7258 - val_acc: 0.6516
Epoch 3/10
3774/3774 [==============================] - 0s 86us/sample - loss: 2.5522 - acc: 0.6471 - val_loss: 2.5828 - val_acc: 0.7344
Epoch 4/10
3774/3774 [==============================] - 0s 87us/sample - loss: 2.6677 - acc: 0.6672 - val_loss: 1.3398 - val_acc: 0.7102
Epoch 5/10
3774/3774 [==============================] - 0s 86us/sample - loss: 2.7825 - acc: 0.6709 - val_loss: 2.9961 - val_acc: 0.6097
Epoch 6/10
3774/3774 [==============================]

In [5]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test score is {score[0]}")

Test score is 2.4375785453003123


# Grid Search

In [7]:
inputs = X_train.shape[1]

def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

c:\users\han-chung lee\.virtualenvs\ds-unit-4-sprint-2-neural-networks-v4qq6gjw\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Best: 0.74920125802358 using {'batch_size': 60, 'epochs': 10}
Means: 0.7364217241605123, Stdev: 0.007691842480224516 with: {'batch_size': 10, 'epochs': 10}
Means: 0.668264110883077, Stdev: 0.07671059525642886 with: {'batch_size': 20, 'epochs': 10}
Means: 0.6308129131793976, Stdev: 0.17683958558024365 with: {'batch_size': 40, 'epochs': 10}
Means: 0.74920125802358, Stdev: 0.026456701041323683 with: {'batch_size': 60, 'epochs': 10}
Means: 0.6661341786384583, Stdev: 0.017210496789684816 with: {'batch_size': 80, 'epochs': 10}
Means: 0.6173233886559805, Stdev: 0.10908838321424377 with: {'batch_size': 100, 'epochs': 10}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?